# **00 - Introduction to SQL and Database Management for Marketing Analytics**

**_Please follow these steps (Ignore if you've done it already):_**

1. **Go to File -> Save a copy in Drive.**
    - This will create a duplicate of the notebook directly within your own Google Drive account.

2. **Work on the copied notebook.**
    - You can now modify the code, add notes, and experiment within your own copy.

---

<br>


### **Understanding SQL**
- **Definition**: **SQL (Structured Query Language) is a specialized language used for interacting with databases**. It is not a full programming language but a data sublanguage designed for defining and manipulating database content.
- **History**: **Developed by IBM in the late 1970s**, SQL has been standardized by ANSI and ISO, evolving through versions like SQL-92, SQL:1999, SQL:2003, to SQL:2008, integrating features such as object-oriented concepts and XML support.

<br>


### **SQL in Practice**
- **Implementation**: SQL commands are **executed via text processors or through graphical interfaces in database management systems (DBMS)** like Microsoft Access, SQL Server, Oracle, and MySQL.
- **Relevance**: Knowing SQL **enhances capabilities in database management**, even in environments that provide graphical tools, by allowing dynamic SQL statement generation and comprehensive database manipulation.

<br>

### **Some SQL Categories**
- **DDL (Data Definition Language)**: Used for **defining database structures**.
- **DML (Data Manipulation Language)**: Enables **querying and modifying database data**. Includes SQL views for creating predefined queries.

<br>

### **Example Database: Wedgewood Pacific Corporation (WPC)**
- **Overview**: **WPC, a company with a diverse departmental structure**, utilizes a database containing information on employees, departments, projects, and assets.
- **Database Relations**:
  - **DEPARTMENT**: Contains basic details about departments.
  - **EMPLOYEE**: Lists employee details, linking to departments.
  - **PROJECT**: Details projects, including departmental association and duration.
  - **ASSIGNMENT**: Tracks employee hours on projects, linking employees to specific tasks.
- **Key Concepts**:
  - Primary and Foreign Keys: Essential for linking tables and ensuring data integrity.
  - Referential Integrity: Ensures foreign keys accurately reference corresponding primary keys in related tables.

<br>


### **Practical Application**
- **Real-World Relevance**: Understanding and manipulating these database structures through **SQL is crucial for marketing analytics**, offering insights into organizational operations, employee contributions, and project management.
- **Analytical Rigor**: Through SQL, marketing students can **develop a foundational understanding of data structure, relational database management, and analytical skills necessary for extracting actionable insights** from complex data sets.


## -> Data Types in SQL and SQLite3 for Marketing Analytics

Understanding data types is fundamental for marketing analysts to effectively structure and query data within a database. Different SQL systems support various data types, but here we'll focus on common SQL data types and those specific to SQLite3.

**Common SQL Data Types:**
- **Numeric:** Integers (`INT`), decimals (`DECIMAL`), floating-point (`FLOAT`), and auto-incrementing numbers (`IDENTITY` in SQL Server, `AUTOINCREMENT` in SQLite for `INTEGER`).
- **Character:** Fixed-length (`CHAR`) and variable-length strings (`VARCHAR`), useful for storing text data like names, addresses, or descriptions.
- **Date and Time:** Date (`DATE`), time (`TIME`), and both (`DATETIME`), essential for tracking event times, campaign durations, or customer transactions.
- **Binary:** Binary data types (`BINARY`, `VARBINARY`) for storing binary data such as images or file blobs.
- **Boolean:** A `BOOLEAN` type representing true or false, practical for flags and switches in marketing campaigns.

**SQLite3 Specific Data Types:**
- **INTEGER:** The primary integer data type in SQLite. It can be auto-incremented.
- **TEXT:** Equivalent to `VARCHAR` in SQL; used to store text data.
- **BLOB:** For binary data; SQLite does not distinguish between `BLOB` and text data types, and it stores all data exactly as provided.
- **REAL:** The floating-point numeric data type in SQLite, similar to `FLOAT` or `DOUBLE` in SQL.
- **NUMERIC:** General numeric type that can store `INT`, `REAL`, `DECIMAL`, and boolean values. SQLite uses dynamic typing and can store any data type in `NUMERIC`.

**Real-World Application in Marketing:**
- **INTEGER:** Ideal for unique customer IDs or transaction numbers.
- **TEXT:** Used for storing customer feedback or product descriptions.
- **REAL and NUMERIC:** Perfect for financial calculations like budgeting, ROI analysis, or sales data.
- **DATE and TIME:** Vital for campaign planning, tracking sale periods, or customer purchase history.

**Type Affinity in SQLite3:**
SQLite3 uses a concept called "type affinity" on column definitions, which is a recommended type rather than a strict one. This flexibility allows the database to store any type of data in any type of column but recommends a particular data type for each column based on the column's declared type.

**Best Practices for Marketers:**
- Choose data types that best reflect the nature of the data and its use in analysis.
- Use appropriate types for dates and numbers to ensure accurate sorting, filtering, and calculations.
- Store data in the most efficient data type to optimize database size and performance.


# **L01 - Data Definition Language (DDL)**

In [7]:
import sqlite3
import pandas as pd

def print_db_info(db_file):
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # Fetch and print the list of tables in the database
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
    tables = cursor.fetchall()
    print("The database contains the following tables:")
    for table in tables:
        print(table[0])

    print("\n" + "="*100 + "\n")

    # For each table, print the table schema in pandas info style
    # and display the first few rows like pd.head(), including foreign key info
    print("\nPrinting tables in pandas info style and displaying first few rows like pd.head():")
    for table in tables:
        table_name = table[0]
        print(f"\n\n\n###---DB Table Name---###: {table_name}")

        # Printing table schema
        cursor.execute(f"PRAGMA table_info({table_name});")
        table_info = cursor.fetchall()
        column_names = [row[1] for row in table_info]
        data_types = [row[2] for row in table_info]
        can_be_null = ['Yes' if row[3] == 0 else 'No' for row in table_info]
        default_values = [row[4] for row in table_info]
        primary_keys = ['Yes' if row[5] == 1 else 'No' for row in table_info]

        # Fetch foreign key information
        cursor.execute(f"PRAGMA foreign_key_list({table_name});")
        fk_info = cursor.fetchall()
        fk_columns = [row[3] for row in fk_info]  # Column name in the table that is a foreign key

        # Mark primary and foreign keys
        keys_info = []
        for column in column_names:
            if column in fk_columns:
                key_type = 'Foreign'
            elif primary_keys[column_names.index(column)] == 'Yes':
                key_type = 'Primary'
            else:
                key_type = 'No'
            keys_info.append(key_type)

        table_dict = {
            "Column": column_names,
            "Type": data_types,
            "Can be Null": can_be_null,
            "Default": default_values,
            "Key": keys_info  # Updated to include both primary and foreign keys
        }
        print(pd.DataFrame(table_dict))

        # Displaying the first few rows of each table like pd.head()
        try:
            df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
            print("\n-*Rows of Table*-:")
            print(df)
        except Exception as e:
            print(f"Could not display rows for {table_name}: {e}")


    print("\n" + "="*100 + "\n")

    conn.close()

## -> L01A ->  SQL for Data Definition in Marketing Analytics

**Understanding SQL DDL (Data Definition Language)**
In database management, SQL DDL is crucial for creating the structure of databases and tables which are essential for storing marketing data. It includes statements like `CREATE TABLE`, which is used to establish new tables with specific structures.

**Table Creation Basics**
- A table is defined with the `CREATE TABLE` statement followed by the table name and a set of column definitions.
- Each column has a name, a data type (like `Char`, `Int`, `VarChar`, `DateTime`), and constraints (like `PRIMARY KEY`, `NOT NULL`, `NULL`, `UNIQUE`).
- `DEFAULT` values can be set for columns to be used when no other value is provided.

**Practical Example in Marketing**
- `DEPARTMENT` table might store various marketing departments with a budget code and contact information.
- `EMPLOYEE` table holds details about marketing team members, ensuring unique email addresses with the `UNIQUE` constraint.
- `PROJECT` table tracks marketing projects, setting a default value for maximum hours using `DEFAULT`.

**Column Data Types and Constraints**
- `Char` and `VarChar` handle text data, the latter being more space-efficient as it stores variable-length data.
- `Numeric(8,2)` indicates a numeric value with 2 decimal places.
- Constraints ensure data integrity; `PRIMARY KEY` identifies unique rows, `NOT NULL` ensures a value must be entered, and `UNIQUE` prevents duplicates.

**Real-World Relevance**
- In a marketing database, ensuring unique email addresses prevents communication mishaps.
- Defaulting to a specific department can streamline data entry for new employees focused on a primary area like 'Human Resources'.

**Data Types and DBMS Specifics**
- While standard SQL data types are used, each DBMS might have additional types. For marketing analytics, choosing the right data type can impact data storage and query efficiency.


**Python Code with sqlite3**
- This Python code uses `sqlite3`, which is typically included in Python standard libraries, to execute SQL DDL statements, mimicking the actions described in the provided SQL content.
  - Marketing students should understand how to translate SQL DDL into Python code to manage their marketing databases.


## -> L01B -> SQL Table Constraints in Marketing Analytics

**Defining Primary Keys**
- Primary keys uniquely identify each record in a table and are essential for relational database integrity, crucial in marketing analytics for tracking unique customer data, campaign elements, or sales transactions.
- Table constraints with `CONSTRAINT` keyword provide a standardized way of defining primary keys and other constraints, enhancing database structure clarity.

**Advantages of Table Constraints**
- **Composite Keys:** Necessary for defining a primary key across multiple columns, useful for linking complex marketing data like assignment of employees to multiple projects.
- **Naming Constraints:** Offers control over constraint names, aiding in database administration and maintenance.
- **Surrogate Keys:** Enables the use of surrogate keys, which are artificial keys often used when natural primary keys are not available or practical.

**Real-World Application in Marketing**
- For example, in a marketing context, a composite key in an `ASSIGNMENT` table could link marketing projects to team members, ensuring data about who is working on what is organized and accessible.

<br>

**Python Code with sqlite3**

**Note:** SQLite does not support the `IDENTITY` property used in SQL Server for auto-incrementing keys. In SQLite, the `AUTOINCREMENT` keyword can be used in a similar fashion, but it is not necessary unless you need a guarantee that the auto-incremented values are strictly monotonically increasing. SQLite by default will auto-increment integer primary keys without this keyword.


## -> L01 C -> Foreign Key Constraints in Marketing Databases

**Introduction to Foreign Keys**
Foreign keys are a fundamental aspect of relational databases, ensuring referential integrity between tables. They create a link between two tables, ensuring that the data corresponds to existing, valid data in another table.

**Defining Foreign Keys with Constraints**
- A foreign key constraint is defined in the context of a table and specifies that a column or a set of columns from one table references the primary key columns of another table.
- This establishes a relationship between the two tables, which is essential for joining data in queries, a common task in marketing analytics for merging customer data with purchase orders, for example.

**Actions on Update and Delete**
- `ON UPDATE CASCADE`: Automatically updates the foreign key column values in the referencing table when the primary key value is updated in the referenced table.
- `ON DELETE CASCADE`: Deletes all rows in the referencing table that correspond to the deleted row in the referenced table.
- `NO ACTION`: Specifies that if the referenced row is updated or deleted, no action will be taken in the referencing table. This is important when the referenced data should not change, like a historical log of marketing campaigns.

**Practical Application in Marketing**
- Foreign key constraints are used to maintain consistent data across different marketing efforts. For example, if a department's name changes, all employee records will automatically update, reflecting this change in the `EMPLOYEE` table.

<br>

### **Python Code with sqlite3**
**Note:** SQLite's syntax differs slightly from other SQL dialects. For instance, SQLite uses `AUTOINCREMENT` keyword for automatically incrementing primary keys, and unlike some SQL databases, `ON UPDATE CASCADE

` and `ON DELETE CASCADE` must be declared at the table level, not the column level. This script reflects SQLite's conventions and sets up a marketing database schema with appropriate foreign key constraints for maintaining data integrity.

Understanding these relationships and constraints is vital for marketing professionals as it affects data management, reporting, and the accuracy of insights derived from marketing analytics.

In [8]:
import sqlite3
import sys

# Suppress start-up warnings
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

# Ensure sqlite3 is available, as Google Colab should already include it.
try:
    import sqlite3
except ImportError:
    !pip install sqlite3

# Establishing a connection to the SQLite database
conn = sqlite3.connect('marketing_db.sqlite')
cursor = conn.cursor()


cursor.execute('DROP TABLE IF EXISTS ASSIGNMENT')
cursor.execute('DROP TABLE IF EXISTS PROJECT')
cursor.execute('DROP TABLE IF EXISTS EMPLOYEE')
cursor.execute('DROP TABLE IF EXISTS DEPARTMENT')

cursor.execute('''CREATE TABLE DEPARTMENT (
   DepartmentName CHAR(35) NOT NULL,
   BudgetCode CHAR(30) NOT NULL,
   OfficeNumber CHAR(15) NOT NULL,
   Phone CHAR(12) NOT NULL,
   CONSTRAINT DEPARTMENT_PK PRIMARY KEY (DepartmentName)
)''')

cursor.execute('''CREATE TABLE EMPLOYEE (
   EmployeeNumber INTEGER PRIMARY KEY,
   FirstName CHAR(25) NOT NULL,
   LastName CHAR(25) NOT NULL,
   Department CHAR(35) NOT NULL DEFAULT 'Human Resources',
   Phone CHAR(12),
   Email VARCHAR(100) NOT NULL UNIQUE,
   CONSTRAINT EMP_DEPART_FK FOREIGN KEY (Department)
      REFERENCES DEPARTMENT (DepartmentName)
         ON UPDATE CASCADE
)''')

cursor.execute('''CREATE TABLE PROJECT (
   ProjectID INTEGER PRIMARY KEY,
   ProjectName CHAR(50) NOT NULL,
   Department CHAR(35) NOT NULL,
   MaxHours NUMERIC(8,2) NOT NULL DEFAULT 100,
   StartDate DATETIME,
   EndDate DATETIME,
   CONSTRAINT PROJ_DEPART_FK FOREIGN KEY (Department)
      REFERENCES DEPARTMENT (DepartmentName)
         ON UPDATE CASCADE
)''')

cursor.execute('''CREATE TABLE ASSIGNMENT (
   ProjectID INTEGER NOT NULL,
   EmployeeNumber INTEGER NOT NULL,
   HoursWorked NUMERIC(6,2),
   CONSTRAINT ASSIGNMENT_PK PRIMARY KEY (ProjectID, EmployeeNumber),
   CONSTRAINT ASSIGN_PROJ_FK FOREIGN KEY (ProjectID)
      REFERENCES PROJECT (ProjectID)
         ON DELETE CASCADE,
   CONSTRAINT ASSIGN_EMP_FK FOREIGN KEY (EmployeeNumber)
      REFERENCES EMPLOYEE (EmployeeNumber)
        ON DELETE NO ACTION
)''')

conn.commit()
conn.close()
print_db_info("marketing_db.sqlite")

The database contains the following tables:
DEPARTMENT
EMPLOYEE
PROJECT
ASSIGNMENT



Printing tables in pandas info style and displaying first few rows like pd.head():



###---DB Table Name---###: DEPARTMENT
           Column      Type Can be Null Default      Key
0  DepartmentName  CHAR(35)          No    None  Primary
1      BudgetCode  CHAR(30)          No    None       No
2    OfficeNumber  CHAR(15)          No    None       No
3           Phone  CHAR(12)          No    None       No

-*Rows of Table*-:
Empty DataFrame
Columns: [DepartmentName, BudgetCode, OfficeNumber, Phone]
Index: []



###---DB Table Name---###: EMPLOYEE
           Column          Type Can be Null            Default      Key
0  EmployeeNumber       INTEGER         Yes               None  Primary
1       FirstName      CHAR(25)          No               None       No
2        LastName      CHAR(25)          No               None       No
3      Department      CHAR(35)          No  'Human Resources'  Foreign
4

# **L02 - Data Manipulation Language (DML)**

## -> L02A -> Understanding SQL Data Modification
- SQL Data Manipulation Language (DML) is used for querying and modifying table data.
- There are three main operations: `INSERT`, `UPDATE`, and `DELETE`.
- We focus on `INSERT` to populate database tables with data.
### Inserting Data into Tables
- The `INSERT INTO` statement adds data to a table.
- If all column data is provided, use: `INSERT INTO table_name VALUES(...);`.
- If not all columns are provided, specify column names: `INSERT INTO table_name (column1, column2) VALUES(...);`.
- The order of values must match the order of the specified columns.
- All `NOT NULL` columns must have a value; otherwise, a `NULL` value is inserted.
### SQL Comments
- SQL comments are enclosed in `/ ... /` and are ignored during execution.
- They are useful for documenting SQL statements.
### Practical Examples
- Inserting department data: `INSERT INTO DEPARTMENT VALUES('Marketing', 'BC-700-10', 'BLDG02-200', '360-287-8700');`
- Inserting employee data with missing phone number: `INSERT INTO EMPLOYEE(FirstName, LastName, Department, Email) VALUES('James', 'Nestor', 'InfoSystems', 'James.Nestor@wpc.com');`
- Inserting project data with a `NULL` end date: `INSERT INTO PROJECT (ProjectName, Department, MaxHours, StartDate) VALUES('2012 Q4 Portfolio Analysis', 'Finance', 140.00, '05-OCT-12');`
### Python Code with sqlite3

In [9]:
import sqlite3
import sys

# Suppress start-up warnings
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

# Ensure sqlite3 is available, as Google Colab should already include it.
try:
    import sqlite3
except ImportError:
    !pip install sqlite3

# Establishing a connection to the SQLite database
conn = sqlite3.connect('marketing_db.sqlite')
cursor = conn.cursor()

##
# Insert data into DEPARTMENT table
department_data = [
  ('Administration', 'BC-100-10', 'BLDG01-300', '360-285-8100'),
  ('Legal', 'BC-200-10', 'BLDG01-200', '360-285-8200'),
  ('Accounting', 'BC-300-10', 'BLDG01-100', '360-285-8300'),
  ('Finance', 'BC-400-10', 'BLDG01-140', '360-285-8400'),
  ('Human Resources', 'BC-500-10', 'BLDG01-180', '360-285-8500'),
  ('Production', 'BC-600-10', 'BLDG02-100', '360-287-8600'),
  ('Marketing', 'BC-700-10', 'BLDG02-200', '360-287-8700'),
  ('InfoSystems', 'BC-800-10', 'BLDG02-270', '360-287-8800')
]
for dept in department_data:
  cursor.execute('INSERT INTO DEPARTMENT (DepartmentName, BudgetCode, OfficeNumber, Phone) VALUES (?, ?, ?, ?)', dept)

##
# Insert data into EMPLOYEE table
employee_data = [
  (1, 'Mary', 'Jacobs', 'Administration', '360-285-8110', 'Mary.Jacobs@wpc.com'),
  (2, 'Rosalie', 'Jackson', 'Administration', '360-285-8120', 'Rosalie.Jackson@wpc.com'),
  (3, 'Richard', 'Bandalone', 'Legal', '360-285-8210', 'Richard.Bandalone@wpc.com'),
  (4, 'Tom', 'Caruthers', 'Accounting', '360-285-8310', 'Tom.Caruthers@wpc.com'),
  (5, 'Heather', 'Jones', 'Accounting', '360-285-8320', 'Heather.Jones@wpc.com'),
  (6, 'Mary', 'Abernathy', 'Finance', '360-285-8410', 'Mary.Abernathy@wpc.com'),
  (7, 'George', 'Smith', 'Human Resources', '360-285-8510', 'George.Smith@wpc.com'),
  (8, 'Tom', 'Jackson', 'Production', '360-287-8610', 'Tom.Jackson@wpc.com'),
  (9, 'George', 'Jones', 'Production', '360-287-8620', 'George.Jones@wpc.com'),
  (10, 'Ken', 'Numoto', 'Marketing', '360-287-8710', 'Ken.Numoto@wpc.com'),
  (11, 'James', 'Nestor', 'InfoSystems', None,'James.Nestor@wpc.com'),
  (12, 'Rick', 'Brown', 'InfoSystems', '360-287-8820', 'Rick.Brown@wpc.com')
]
for emp in employee_data:
  cursor.execute('INSERT INTO EMPLOYEE (EmployeeNumber, FirstName, LastName,  Department, Phone, Email) VALUES (?, ?, ?, ?, ?, ?)', emp)

##
# Insert data into PROJECT table
project_data = [
  (1000, '2012 Q3 Product Plan', 'Marketing', 135.00, '2012-05-10', '2012-06-15'),
  (1100, '2012 Q3 Portfolio Analysis', 'Finance', 120.00, '2012-07-05', '2012-07-25'),
  (1200, '2012 Q3 Tax Preparation', 'Accounting', 145.00, '2012-08-10', '2012-10-25'),
  (1300, '2012 Q4 Product Plan', 'Marketing', 150.00, '2012-08-10', '2012-09-15'),
  (1400, '2012 Q4 Portfolio Analysis', 'Finance', 140.00, '2012-10-05', None)
]

for proj in project_data:
  cursor.execute('INSERT INTO PROJECT (ProjectID, ProjectName, Department, MaxHours, StartDate, EndDate) VALUES (?, ?, ?, ?, ?, ?)', proj)


##
# Insert data into ASSIGNMENT table
assignment_data = [
  (1000, 1, 30.0),
  (1000, 8, 75.0),
  (1000, 10, 55.0),
  (1100, 4, 40.0),
  (1100, 6, 45.0),
  (1200, 1, 25.0),
  (1200, 2, 20.0),
  (1200, 4, 45.0),
  (1200, 5, 40.0),
  (1300, 1, 35.0),
  (1300, 8, 80.0),
  (1300, 10, 50.0),
  (1400, 4, 15.0),
  (1400, 5, 10.0),
  (1400, 6, 27.5)
]
for assigned in assignment_data:
  cursor.execute('INSERT INTO ASSIGNMENT (ProjectID, EmployeeNumber, HoursWorked) VALUES (?, ?, ?)', assigned)

##
# Commit changes and close the connection
conn.commit()
conn.close()

##
# Predefined function to print database info
print_db_info("marketing_db.sqlite")

The database contains the following tables:
DEPARTMENT
EMPLOYEE
PROJECT
ASSIGNMENT



Printing tables in pandas info style and displaying first few rows like pd.head():



###---DB Table Name---###: DEPARTMENT
           Column      Type Can be Null Default      Key
0  DepartmentName  CHAR(35)          No    None  Primary
1      BudgetCode  CHAR(30)          No    None       No
2    OfficeNumber  CHAR(15)          No    None       No
3           Phone  CHAR(12)          No    None       No

-*Rows of Table*-:
    DepartmentName BudgetCode OfficeNumber         Phone
0   Administration  BC-100-10   BLDG01-300  360-285-8100
1            Legal  BC-200-10   BLDG01-200  360-285-8200
2       Accounting  BC-300-10   BLDG01-100  360-285-8300
3          Finance  BC-400-10   BLDG01-140  360-285-8400
4  Human Resources  BC-500-10   BLDG01-180  360-285-8500
5       Production  BC-600-10   BLDG02-100  360-287-8600
6        Marketing  BC-700-10   BLDG02-200  360-287-8700
7      InfoSystems  BC-800-10

## -> L02B -> **Marketing Database Analytics: Understanding SQL Queries**
In this class, we will explore the foundational SQL SELECT/FROM/WHERE framework, which is essential for querying databases in marketing analytics. We'll use Python's `sqlite3` module to interact with our `marketing_db.sqlite` database.
### **The SELECT/FROM/WHERE Framework**
The SQL SELECT command is used to specify the columns you want to retrieve from a database table. The FROM clause indicates the table from which to select, and the WHERE clause filters the results based on a condition.
- Basic Query Structure: `SELECT ColumnNames FROM TableName WHERE SomeConditionExists;`
- Reading Specific Columns: To read certain columns from a table, list them after SELECT.
- Resulting Relation: The result of a SELECT statement is always a relation, which can be a table or a single value.
### **Reading Specified Columns from a Single Table**
Let's look at how to query specific columns from a table called `PROJECT`.
- Example Query: `SELECT ProjectName, Department, MaxHours FROM PROJECT;`
- Order Matters: The order of columns in the SELECT statement determines their order in the result set.
### **Eliminating Duplicate Rows**
By default, SQL does not eliminate duplicate rows. To ensure uniqueness, use the `DISTINCT` keyword.
- Example Query with DISTINCT: `SELECT DISTINCT Department FROM PROJECT;`
### **Python Code with sqlite3**
Below is the Python `sqlite3` equivalent of the SQL provided, including the installation of necessary libraries for a Google Colab environment.


In [10]:
import sqlite3
import pandas as pd
import re

def transform_right_to_left_join(query):
    """
    Transforms SQL queries with RIGHT JOINs to use LEFT JOINs instead,
    rearranging the tables and join conditions appropriately.
    """
    # Regular expression to find and capture parts of the RIGHT JOIN query for transformation
    right_join_pattern = re.compile(
        r'SELECT (.+?)\sFROM \((.+?) AS (.+?) JOIN (.+?) AS (.+?)\sON (.+?)\)\sRIGHT JOIN (.+?) AS (.+?)\sON (.+?)\sORDER BY (.+);',
        re.IGNORECASE | re.DOTALL
    )

    # Replacement pattern to transform the query
    replacement_pattern = r'SELECT \1 FROM \7 AS \8 LEFT JOIN (\4 AS \5 JOIN \2 AS \3 ON \6) ON \9 ORDER BY \10;'

    # Replace the matched pattern in the query
    transformed_query = right_join_pattern.sub(replacement_pattern, query)

    return transformed_query

def SQLite3_Practice(database_file, query):
    """Executes an SQLite3 query on a database, reads results into a pandas DataFrame, and prints the output.

    Args:
        database_file (str): Path to the SQLite3 database file.
        query (str): The SQL query to execute.
    """
    # Transform RIGHT JOINs to LEFT JOINs in the query
    query = transform_right_to_left_join(query)

    try:
        # Connect to the database
        conn = sqlite3.connect(database_file)
        cursor = conn.cursor()

        # Execute the query
        cursor.execute(query)

        # Fetch the results
        rows = cursor.fetchall()

        # Extract column names
        columns = [desc[0] for desc in cursor.description]

        # Create and print the DataFrame
        df = pd.DataFrame(rows, columns=columns)

        # Adjust the index to start from 1 instead of 0
        df.index = df.index + 1

        print(df)

    except sqlite3.Error as e:
        print("Error:", e)
    finally:
        # Ensure the connection is closed even if an error occurs
        if conn:
            conn.close()

# Example usage
database_file = 'marketing_db.sqlite'  # Replace with your database file

### ///***Reading Specified Columns from a Single Table***
The following SQL statement queries (reads) three of the six columns of the PROJECT table:

In [11]:
# Query 1
query = """
SELECT ProjectName, Department, MaxHours FROM PROJECT
"""
SQLite3_Practice(database_file, query)

                  ProjectName  Department  MaxHours
1        2012 Q3 Product Plan   Marketing       135
2  2012 Q3 Portfolio Analysis     Finance       120
3     2012 Q3 Tax Preparation  Accounting       145
4        2012 Q4 Product Plan   Marketing       150
5  2012 Q4 Portfolio Analysis     Finance       140


In [12]:
# Query 2
query = """
SELECT DISTINCT Department FROM PROJECT
"""
SQLite3_Practice(database_file, query)

   Department
1   Marketing
2     Finance
3  Accounting


In [13]:
# Query 3 - The next SQL statement obtains only the Department column from the PROJECT table:
query = """
SELECT Department
FROM PROJECT;
"""
SQLite3_Practice(database_file, query)


   Department
1   Marketing
2     Finance
3  Accounting
4   Marketing
5     Finance


In [14]:
# Query 4 - If you want the DBMS to check for and eliminate duplicate rows, you must use the DISTINCT keyword, as follows:
query = """
/* *** SQL-QUERY-CH03-04 *** */
SELECT DISTINCT Department
FROM PROJECT;
"""
SQLite3_Practice(database_file, query)


   Department
1   Marketing
2     Finance
3  Accounting


In [15]:
# Query 5 - If you want the DBMS to check for and eliminate duplicate rows, you must use the DISTINCT keyword, as follows:
query = """
/* *** SQL-QUERY-CH03-04 *** */
SELECT DISTINCT Department
FROM PROJECT;
"""
SQLite3_Practice(database_file, query)


   Department
1   Marketing
2     Finance
3  Accounting


### ///***Reading Specified Rows from a Single Table***

In [16]:
# Query 5 - If you want the DBMS to check for and eliminate duplicate rows,
# you must use the DISTINCT keyword, as follows:

query = """
/* *** SQL-QUERY-CH03-05 *** */
SELECT ProjectID, ProjectName, Department, MaxHours,
StartDate, EndDate
FROM PROJECT
WHERE Department = 'Finance';
"""
SQLite3_Practice(database_file, query)


   ProjectID                 ProjectName Department  MaxHours   StartDate  \
1       1100  2012 Q3 Portfolio Analysis    Finance       120  2012-07-05   
2       1400  2012 Q4 Portfolio Analysis    Finance       140  2012-10-05   

      EndDate  
1  2012-07-25  
2        None  


In [17]:
# Query 6 - selects all columns from PROJECT where the value of the MaxHours
# column is greater than 135

query = """
/* *** SQL-QUERY-CH03-07 *** */
SELECT *
FROM PROJECT
WHERE MaxHours > 135;
"""
SQLite3_Practice(database_file, query)

   ProjectID                 ProjectName  Department  MaxHours   StartDate  \
1       1200     2012 Q3 Tax Preparation  Accounting       145  2012-08-10   
2       1300        2012 Q4 Product Plan   Marketing       150  2012-08-10   
3       1400  2012 Q4 Portfolio Analysis     Finance       140  2012-10-05   

      EndDate  
1  2012-10-25  
2  2012-09-15  
3        None  


In [18]:
# Query 7 - the following query determines which projects are sponsored by the finance department and are allocated a maximum number of hours
# greater than 135
query = """
/* *** SQL-QUERY-CH03-08 *** */
SELECT *
FROM PROJECT
WHERE Department = 'Finance' AND MaxHours > 135;
"""
SQLite3_Practice(database_file, query)

   ProjectID                 ProjectName Department  MaxHours   StartDate  \
1       1400  2012 Q4 Portfolio Analysis    Finance       140  2012-10-05   

  EndDate  
1    None  


### ///***Reading Specified Columns and Specified Rows from a Single Table***

In [19]:
# Query 8 - to obtain only the FirstName, LastName, Phone, and Department values of employees in the accounting department, you use:

query = """
/* *** SQL-QUERY-CH03-08 *** */
SELECT *
FROM PROJECT
WHERE Department = 'Finance' AND MaxHours > 135;
"""
SQLite3_Practice(database_file, query)

   ProjectID                 ProjectName Department  MaxHours   StartDate  \
1       1400  2012 Q4 Portfolio Analysis    Finance       140  2012-10-05   

  EndDate  
1    None  


In [20]:
# Query 9 - to obtain only the FirstName, LastName, Phone, and Department values of employees
# in the accounting department, you use:

query = """
/* *** SQL-QUERY-CH03-09 *** */
SELECT FirstName, LastName, Phone, Department
FROM EMPLOYEE
WHERE Department = 'Accounting';
"""
SQLite3_Practice(database_file, query)

  FirstName   LastName         Phone  Department
1       Tom  Caruthers  360-285-8310  Accounting
2   Heather      Jones  360-285-8320  Accounting


In [21]:
# Query 10 - the following query uses the AND keyword to ask for employees that
# work in accounting and have the phone number 360-285-8310:
query = """
/* *** SQL-QUERY-CH03-10 *** */
SELECT FirstName, LastName, Phone, Department
FROM EMPLOYEE
WHERE Department = 'Accounting'
AND Phone = '360-285-8310';
"""
SQLite3_Practice(database_file, query)

  FirstName   LastName         Phone  Department
1       Tom  Caruthers  360-285-8310  Accounting


In [22]:
# Query 11 - the following query uses the OR keyword to ask for employees that work in
# accounting or have the phone number 360-285-8410:
query = """
/* *** SQL-QUERY-CH03-11 *** */
SELECT FirstName, LastName, Phone, Department
FROM EMPLOYEE
WHERE Department = 'Accounting'
OR Phone = '360-285-8410';
"""
SQLite3_Practice(database_file, query)

  FirstName   LastName         Phone  Department
1       Tom  Caruthers  360-285-8310  Accounting
2   Heather      Jones  360-285-8320  Accounting
3      Mary  Abernathy  360-285-8410     Finance


In [23]:
# Query 12 - In this query, a row will be displayed if it has a Department value equal to Accounting,
# Finance, or Marketing.
query = """
/* *** SQL-QUERY-CH03-12 *** */
SELECT FirstName, LastName, Phone, Department
FROM EMPLOYEE
WHERE Department IN ('Accounting', 'Finance',
'Marketing');
"""
SQLite3_Practice(database_file, query)

  FirstName   LastName         Phone  Department
1       Tom  Caruthers  360-285-8310  Accounting
2   Heather      Jones  360-285-8320  Accounting
3      Mary  Abernathy  360-285-8410     Finance
4       Ken     Numoto  360-287-8710   Marketing


In [24]:
# Query 13 - To select rows that do not have any of these Department values, you would use the
# NOT keyword in the NOT IN phrase, as follows:
query = """
/* *** SQL-QUERY-CH03-13 *** */
SELECT FirstName, LastName, Phone, Department
FROM EMPLOYEE
WHERE Department NOT IN ('Accounting', 'Finance',
'Marketing');
"""
SQLite3_Practice(database_file, query)

  FirstName   LastName         Phone       Department
1      Mary     Jacobs  360-285-8110   Administration
2   Rosalie    Jackson  360-285-8120   Administration
3   Richard  Bandalone  360-285-8210            Legal
4    George      Smith  360-285-8510  Human Resources
5       Tom    Jackson  360-287-8610       Production
6    George      Jones  360-287-8620       Production
7     James     Nestor          None      InfoSystems
8      Rick      Brown  360-287-8820      InfoSystems


### ///***Ranges, Wildcards, and Nulls in WHERE Clauses***

In [25]:
# Query 14 - WHERE clauses can refer to ranges of values and partial values. The BETWEEN keyword
# is used for ranges of values. For example, the statement:
query = """
/* *** SQL-QUERY-CH03-14 *** */
SELECT FirstName, LastName, Phone, Department
FROM EMPLOYEE
WHERE EmployeeNumber BETWEEN 2 AND 5;
"""
SQLite3_Practice(database_file, query)

  FirstName   LastName         Phone      Department
1   Rosalie    Jackson  360-285-8120  Administration
2   Richard  Bandalone  360-285-8210           Legal
3       Tom  Caruthers  360-285-8310      Accounting
4   Heather      Jones  360-285-8320      Accounting


In [26]:
# Query 15 - Note that the SQL keyword BETWEEN includes the end points, and thus SQLQUERY-
# CH03-14 is equivalent to the following query, which uses the SQL comparison
# operators >5 (greater than or equal to) and <5 (less than or equal to):
query = """
/* *** SQL-QUERY-CH03-15 *** */
SELECT FirstName, LastName, Phone, Department
FROM EMPLOYEE
WHERE EmployeeNumber >= 2
AND EmployeeNumber <= 5;
"""
SQLite3_Practice(database_file, query)

  FirstName   LastName         Phone      Department
1   Rosalie    Jackson  360-285-8120  Administration
2   Richard  Bandalone  360-285-8210           Legal
3       Tom  Caruthers  360-285-8310      Accounting
4   Heather      Jones  360-285-8320      Accounting


In [27]:
# Query 16 - In the following query, LIKE is used with the underscore symbol to find values that fit
# a pattern:
query = """
/* *** SQL-QUERY-CH03-16 *** */
SELECT *
FROM PROJECT
WHERE ProjectName LIKE '2012 Q_ Portfolio Analysis';
"""
SQLite3_Practice(database_file, query)

   ProjectID                 ProjectName Department  MaxHours   StartDate  \
1       1100  2012 Q3 Portfolio Analysis    Finance       120  2012-07-05   
2       1400  2012 Q4 Portfolio Analysis    Finance       140  2012-10-05   

      EndDate  
1  2012-07-25  
2        None  


In [28]:
# Query 17 - To find all employees who have a Phone value that begins with 360-287-,
# you can use four underscores to represent any last four digits, as follows:
query = """
/* *** SQL-QUERY-CH03-17 *** */
SELECT *
FROM EMPLOYEE
WHERE Phone LIKE '360-287-____';
"""
SQLite3_Practice(database_file, query)

   EmployeeNumber FirstName LastName   Department         Phone  \
1               8       Tom  Jackson   Production  360-287-8610   
2               9    George    Jones   Production  360-287-8620   
3              10       Ken   Numoto    Marketing  360-287-8710   
4              12      Rick    Brown  InfoSystems  360-287-8820   

                  Email  
1   Tom.Jackson@wpc.com  
2  George.Jones@wpc.com  
3    Ken.Numoto@wpc.com  
4    Rick.Brown@wpc.com  


In [29]:
# Query 18 - Because the percent sign represents one or more unknown characters,
# another way to write
# the query for employees who have a phone number that starts with 360-287- is:

query = """
/* *** SQL-QUERY-CH03-18 *** */
SELECT *
FROM EMPLOYEE
WHERE Phone LIKE '360-287-%';
"""
SQLite3_Practice(database_file, query)

   EmployeeNumber FirstName LastName   Department         Phone  \
1               8       Tom  Jackson   Production  360-287-8610   
2               9    George    Jones   Production  360-287-8620   
3              10       Ken   Numoto    Marketing  360-287-8710   
4              12      Rick    Brown  InfoSystems  360-287-8820   

                  Email  
1   Tom.Jackson@wpc.com  
2  George.Jones@wpc.com  
3    Ken.Numoto@wpc.com  
4    Rick.Brown@wpc.com  


In [30]:
# Query 19 - If you want to find all the employees who work in departments that end in ing, you can
# use the % character as follows:
query = """
/* *** SQL-QUERY-CH03-19 *** */
SELECT *
FROM EMPLOYEE
WHERE Department LIKE '%ing';
"""
SQLite3_Practice(database_file, query)

   EmployeeNumber FirstName   LastName  Department         Phone  \
1               4       Tom  Caruthers  Accounting  360-285-8310   
2               5   Heather      Jones  Accounting  360-285-8320   
3              10       Ken     Numoto   Marketing  360-287-8710   

                   Email  
1  Tom.Caruthers@wpc.com  
2  Heather.Jones@wpc.com  
3     Ken.Numoto@wpc.com  


In [31]:
# Query 20 - if you want to find all the employees who work in departments that do not end in
# 'ing', you can use the following SQL query:
query = """
SELECT *
FROM EMPLOYEE
WHERE Department NOT LIKE '%ing';
"""
SQLite3_Practice(database_file, query)

   EmployeeNumber FirstName   LastName       Department         Phone  \
1               1      Mary     Jacobs   Administration  360-285-8110   
2               2   Rosalie    Jackson   Administration  360-285-8120   
3               3   Richard  Bandalone            Legal  360-285-8210   
4               6      Mary  Abernathy          Finance  360-285-8410   
5               7    George      Smith  Human Resources  360-285-8510   
6               8       Tom    Jackson       Production  360-287-8610   
7               9    George      Jones       Production  360-287-8620   
8              11     James     Nestor      InfoSystems          None   
9              12      Rick      Brown      InfoSystems  360-287-8820   

                       Email  
1        Mary.Jacobs@wpc.com  
2    Rosalie.Jackson@wpc.com  
3  Richard.Bandalone@wpc.com  
4     Mary.Abernathy@wpc.com  
5       George.Smith@wpc.com  
6        Tom.Jackson@wpc.com  
7       George.Jones@wpc.com  
8       James.Nestor@

In [32]:
# Query 21 - #The following SQL will find the names and departments
# of all employees who have a null value for Phone:
query = """
/* *** SQL-QUERY-CH03-21 *** */
SELECT FirstName, LastName, Phone, Department
FROM EMPLOYEE
WHERE Phone IS NULL;
"""
SQLite3_Practice(database_file, query)

  FirstName LastName Phone   Department
1     James   Nestor  None  InfoSystems


In [33]:
# Query 22 - if you want to find all the employees who do have
# phone numbers, you can use the following SQL query:
query = """
/* *** SQL-QUERY-CH03-22 *** */
SELECT FirstName, LastName, Phone, Department
FROM EMPLOYEE
WHERE Phone IS NOT NULL;
"""
SQLite3_Practice(database_file, query)

   FirstName   LastName         Phone       Department
1       Mary     Jacobs  360-285-8110   Administration
2    Rosalie    Jackson  360-285-8120   Administration
3    Richard  Bandalone  360-285-8210            Legal
4        Tom  Caruthers  360-285-8310       Accounting
5    Heather      Jones  360-285-8320       Accounting
6       Mary  Abernathy  360-285-8410          Finance
7     George      Smith  360-285-8510  Human Resources
8        Tom    Jackson  360-287-8610       Production
9     George      Jones  360-287-8620       Production
10       Ken     Numoto  360-287-8710        Marketing
11      Rick      Brown  360-287-8820      InfoSystems


### ///***Sorting the Results of a Query***

In [34]:
# Query 23 - following will display the names, phone numbers, and departments of all employees, sorted
# by Department:
query = """
/* *** SQL-QUERY-CH03-23 *** */
SELECT FirstName, LastName, Phone, Department
FROM EMPLOYEE
ORDER BY Department;
"""
SQLite3_Practice(database_file, query)

   FirstName   LastName         Phone       Department
1        Tom  Caruthers  360-285-8310       Accounting
2    Heather      Jones  360-285-8320       Accounting
3       Mary     Jacobs  360-285-8110   Administration
4    Rosalie    Jackson  360-285-8120   Administration
5       Mary  Abernathy  360-285-8410          Finance
6     George      Smith  360-285-8510  Human Resources
7      James     Nestor          None      InfoSystems
8       Rick      Brown  360-287-8820      InfoSystems
9    Richard  Bandalone  360-285-8210            Legal
10       Ken     Numoto  360-287-8710        Marketing
11       Tom    Jackson  360-287-8610       Production
12    George      Jones  360-287-8620       Production


In [35]:
# Query 24 -  By default, SQL sorts in ascending order. The ASC keyword and DESC keyword can
# be used to specify ascending and descending order when necessary. Thus, to sort employees
# in descending order by Department, use:
query = """
/* *** SQL-QUERY-CH03-24 *** */
SELECT FirstName, LastName, Phone, Department
FROM EMPLOYEE
ORDER BY Department DESC;
"""
SQLite3_Practice(database_file, query)

   FirstName   LastName         Phone       Department
1        Tom    Jackson  360-287-8610       Production
2     George      Jones  360-287-8620       Production
3        Ken     Numoto  360-287-8710        Marketing
4    Richard  Bandalone  360-285-8210            Legal
5      James     Nestor          None      InfoSystems
6       Rick      Brown  360-287-8820      InfoSystems
7     George      Smith  360-285-8510  Human Resources
8       Mary  Abernathy  360-285-8410          Finance
9       Mary     Jacobs  360-285-8110   Administration
10   Rosalie    Jackson  360-285-8120   Administration
11       Tom  Caruthers  360-285-8310       Accounting
12   Heather      Jones  360-285-8320       Accounting


In [36]:
# Query 25 -  Two or more columns can be used for sorting purposes. To sort the employee names
# and departments first in descending value of Department and then within Department by
# ascending value of LastName, you specify:
query = """
/* *** SQL-QUERY-CH03-25 *** */
SELECT FirstName, LastName, Phone, Department
FROM EMPLOYEE
ORDER BY Department DESC, LastName ASC;
"""
SQLite3_Practice(database_file, query)

   FirstName   LastName         Phone       Department
1        Tom    Jackson  360-287-8610       Production
2     George      Jones  360-287-8620       Production
3        Ken     Numoto  360-287-8710        Marketing
4    Richard  Bandalone  360-285-8210            Legal
5       Rick      Brown  360-287-8820      InfoSystems
6      James     Nestor          None      InfoSystems
7     George      Smith  360-285-8510  Human Resources
8       Mary  Abernathy  360-285-8410          Finance
9    Rosalie    Jackson  360-285-8120   Administration
10      Mary     Jacobs  360-285-8110   Administration
11       Tom  Caruthers  360-285-8310       Accounting
12   Heather      Jones  360-285-8320       Accounting


### ///***SQL Built-in Functions and Calculations***

SQL allows you to calculate values based on the data in the tables. You can use arithmetic formulas, and you can also use SQL built-in functions. SQL includes five built-in functions:
`COUNT`, `SUM`, `AVG`, `MAX`, and `MIN`.

- These functions operate on the results of a SELECT statement.   
  - `COUNT` works regardless of column data type, but `SUM`, `AVG`, `MAX`, and `MIN` operate only on integer, numeric, and other numberoriented columns.
  - `COUNT` and `SUM` sound similar but are different.
    - `COUNT` counts the number of rows in the result, whereas `SUM` totals the set of values of a numeric column.

In [37]:
# Query 26 -  The following SQL statement counts the number of rows in
# the PROJECT table:
query = """
/* *** SQL-QUERY-CH03-26 *** */
SELECT COUNT(*)
FROM PROJECT;
"""
SQLite3_Practice(database_file, query)

   COUNT(*)
1         5


In [38]:
# Query 27 -  You can assign a column name to the result by using the AS keyword:
query = """
/* *** SQL-QUERY-CH03-27 *** */
SELECT COUNT(*) AS NumberOfProjects
FROM PROJECT;
"""
SQLite3_Practice(database_file, query)

   NumberOfProjects
1                 5


In [39]:
# Query 28 & 29 - Consider the following two SELECT statements:
query = """
/* *** SQL-QUERY-CH03-28 *** */
SELECT COUNT(Department) AS NumberOfDepartments
FROM PROJECT;
"""
SQLite3_Practice(database_file, query)

# and:

query = """
/* *** SQL-QUERY-CH03-29 *** */
SELECT COUNT(DISTINCT Department) AS NumberOfDepartments
FROM PROJECT;
"""
SQLite3_Practice(database_file, query)

# The difference in answers occurs because duplicate rows were eliminated in the count of
# the departments in the second SELECT.

   NumberOfDepartments
1                    5
   NumberOfDepartments
1                    3


In [40]:
# Query 30 -  The following is another example of built-in functions:
query = """
/* *** SQL-QUERY-CH03-30 *** */
SELECT MIN(MaxHours) AS MinimumMaxHours,
MAX(MaxHours) AS MaximumMaxHours,
SUM(MaxHours) AS TotalMaxHours
FROM PROJECT
WHERE ProjectID <= 1200;
"""
SQLite3_Practice(database_file, query)

   MinimumMaxHours  MaximumMaxHours  TotalMaxHours
1              120              145            400


---

Standard mathematical calculations can also be done in SQL.
- For example, suppose that all employees at Wedgewood Pacific Corporation are paid $18.50 per hour. Given that each project has a MaxHours value, you might want to calculate a maximum project cost value for each project that is equal to MaxHours multiplied by the hour wage rate.
- You can calculate the needed numbers by using the following query:

In [41]:
# Query 31
query = """
/* *** SQL-QUERY-CH03-31 *** */
SELECT ProjectID, ProjectName, MaxHours,
(18.50 * MaxHours) AS MaxProjectCost
FROM PROJECT;
"""
SQLite3_Practice(database_file, query)

   ProjectID                 ProjectName  MaxHours  MaxProjectCost
1       1000        2012 Q3 Product Plan       135          2497.5
2       1100  2012 Q3 Portfolio Analysis       120          2220.0
3       1200     2012 Q3 Tax Preparation       145          2682.5
4       1300        2012 Q4 Product Plan       150          2775.0
5       1400  2012 Q4 Portfolio Analysis       140          2590.0


### ///***Built-in Functions and Grouping***
In SQL, you can use the GROUP BY clause to group rows by common values.
- This increases the utility of built-in functions because you can apply them to groups of rows.
  - For example, the following statement counts the number of employees in each department:

In [42]:
# Query 34
query = """
/* *** SQL-QUERY-CH03-34 *** */
SELECT Department, Count(*) AS NumberOfEmployees
FROM EMPLOYEE
GROUP BY Department;
"""
SQLite3_Practice(database_file, query)

        Department  NumberOfEmployees
1       Accounting                  2
2   Administration                  2
3          Finance                  1
4  Human Resources                  1
5      InfoSystems                  2
6            Legal                  1
7        Marketing                  1
8       Production                  2


---
We can further restrict the results by using the HAVING clause to apply conditions to the groups that are formed.
- For example, if you want to consider only groups with more than two members, you could specify:

In [43]:
# Query 35
query = """
/* *** SQL-QUERY-CH03-35 *** */
SELECT Department, Count(*) AS NumberOfEmployees
FROM EMPLOYEE
GROUP BY Department
HAVING COUNT(*) > 1;
"""
SQLite3_Practice(database_file, query)

       Department  NumberOfEmployees
1      Accounting                  2
2  Administration                  2
3     InfoSystems                  2
4      Production                  2


---
It is possible to add `WHERE` clauses when using GROUP BY. However, an ambiguity results when this is done.
- If the WHERE condition is applied before the groups are formed, we obtain one result.
  - If, however, the `WHERE` condition is applied after the groups are formed, we get a different result.
    - To resolve this ambiguity, the SQL standard specifies that
when `WHERE` and `GROUP` BY occur together, the `WHERE` condition will be applied first.

For example, consider the following query:

In [44]:
# Query 36A
query = """
/* *** SQL-QUERY-CH03-36 *** */
SELECT Department, Count(*) AS NumberOfEmployees
FROM EMPLOYEE
WHERE EmployeeNumber <= 6
GROUP BY Department
HAVING COUNT(*) > 1;
"""
SQLite3_Practice(database_file, query)

       Department  NumberOfEmployees
1      Accounting                  2
2  Administration                  2


In [45]:
# Query 36B (`HAVING COUNT(*) > 1` removed to show more)
query = """
/* *** SQL-QUERY-CH03-36 *** */
SELECT Department, Count(*) AS NumberOfEmployees
FROM EMPLOYEE
WHERE EmployeeNumber <= 6
GROUP BY Department;
"""
SQLite3_Practice(database_file, query)

       Department  NumberOfEmployees
1      Accounting                  2
2  Administration                  2
3         Finance                  1
4           Legal                  1


## -S06 Continued-

###///***Querying Multiple Tables with Subqueries***

The queries considered so far have involved data from a single table. However, at times, more than one table must be processed to obtain the desired information.
- For example, suppose we want to know the names of all employees who have worked more than 40 hours on any single assignment.
  - The names of employees are stored in the EMPLOYEE table, but the hours they have worked are stored in the ASSIGNMENT table.

In [46]:
# Query 37
# If we knew that employees with EmployeeNumber 8 and 10 have worked more than
# 50 hours on an assignment (which is true), we could obtain their names with
# the following expression:

query = """
/* *** SQL-QUERY-CH03-37 *** */
SELECT FirstName, LastName
FROM EMPLOYEE
WHERE EmployeeNumber IN (8, 10);
"""
SQLite3_Practice(database_file, query)

  FirstName LastName
1       Tom  Jackson
2       Ken   Numoto


In [47]:
# Query 38
# But, according to the problem description, we are not given the employee numbers.
# We can, however, obtain the appropriate employee numbers with the following query:
# But, according to the problem description, we are not given the employee numbers.
# We can, however, obtain the appropriate employee numbers with the following query:

query = """
/* *** SQL-QUERY-CH03-38 *** */
SELECT DISTINCT EmployeeNumber
FROM ASSIGNMENT
WHERE HoursWorked > 50;
"""
SQLite3_Practice(database_file, query)

   EmployeeNumber
1               8
2              10


In [48]:
# Query 39
# Now, we can combine these two SQL statements by using a subquery, as follows:

query = """
/* *** SQL-QUERY-CH03-39 *** */
SELECT FirstName, LastName
FROM EMPLOYEE
WHERE EmployeeNumber IN
(SELECT DISTINCT EmployeeNumber
FROM ASSIGNMENT
WHERE HoursWorked > 50);
"""
SQLite3_Practice(database_file, query)

  FirstName LastName
1       Tom  Jackson
2       Ken   Numoto


Subqueries can be extended to include three, four, or even more levels. Suppose, for example, that you need to know the names of employees who have worked more than 40 hours on an assignment sponsored by the accounting department.

In [49]:
# Query 40
# You can obtain the project IDs of projects sponsored by accounting with:

query = """
/* *** SQL-QUERY-CH03-40 *** */
SELECT ProjectID
FROM PROJECT
WHERE Department = 'Accounting';
"""
SQLite3_Practice(database_file, query)

   ProjectID
1       1200


In [50]:
# Query 41
# You can obtain the employee numbers of employees working more than 40 hours on
# those projects with:

query = """
/* *** SQL-QUERY-CH03-41 *** */
SELECT DISTINCT EmployeeNumber
FROM ASSIGNMENT
WHERE HoursWorked > 40
AND ProjectID IN
(SELECT ProjectID
FROM PROJECT
WHERE Department = 'Accounting');
"""
SQLite3_Practice(database_file, query)

   EmployeeNumber
1               4


In [51]:
# Query 42
# Finally, you can obtain the names of the employees in the preceding
# SQL statement with:

query = """
/* *** SQL-QUERY-CH03-42 *** */
SELECT FirstName, LastName
FROM EMPLOYEE
WHERE EmployeeNumber IN
(SELECT DISTINCT EmployeeNumber
FROM ASSIGNMENT
WHERE HoursWorked > 40
AND ProjectID IN
(SELECT ProjectID
FROM PROJECT
WHERE Department = 'Accounting'));
"""
SQLite3_Practice(database_file, query)

  FirstName   LastName
1       Tom  Caruthers


###///***Querying Multiple Tables with Joins***
---
Subqueries are effective for processing multiple tables, as long as the results come from a single table. If, however, we need to display data from two or more tables, subqueries do not work. We need to use a join operation instead.

The basic idea of a join is to form a new relation by connecting the contents of two or more other relations. Consider the following example:

In [52]:
# Query 43

query = """
/* *** SQL-QUERY-CH03-43 *** */
SELECT FirstName, LastName, HoursWorked
FROM EMPLOYEE, ASSIGNMENT
WHERE EMPLOYEE.EmployeeNumber =
ASSIGNMENT.EmployeeNumber;
"""
SQLite3_Practice(database_file, query)

   FirstName   LastName  HoursWorked
1       Mary     Jacobs         30.0
2        Tom    Jackson         75.0
3        Ken     Numoto         55.0
4        Tom  Caruthers         40.0
5       Mary  Abernathy         45.0
6       Mary     Jacobs         25.0
7    Rosalie    Jackson         20.0
8        Tom  Caruthers         45.0
9    Heather      Jones         40.0
10      Mary     Jacobs         35.0
11       Tom    Jackson         80.0
12       Ken     Numoto         50.0
13       Tom  Caruthers         15.0
14   Heather      Jones         10.0
15      Mary  Abernathy         27.5


In [53]:
# Query 44

query = """
/* *** SQL-QUERY-CH03-44 *** */
SELECT FirstName, LastName, HoursWorked
FROM EMPLOYEE, ASSIGNMENT
WHERE EMPLOYEE.EmployeeNumber =
ASSIGNMENT.EmployeeNumber
ORDER BY EMPLOYEE.EmployeeNumber, ProjectID;
"""
SQLite3_Practice(database_file, query)

   FirstName   LastName  HoursWorked
1       Mary     Jacobs         30.0
2       Mary     Jacobs         25.0
3       Mary     Jacobs         35.0
4    Rosalie    Jackson         20.0
5        Tom  Caruthers         40.0
6        Tom  Caruthers         45.0
7        Tom  Caruthers         15.0
8    Heather      Jones         40.0
9    Heather      Jones         10.0
10      Mary  Abernathy         45.0
11      Mary  Abernathy         27.5
12       Tom    Jackson         75.0
13       Tom    Jackson         80.0
14       Ken     Numoto         55.0
15       Ken     Numoto         50.0


In [54]:
# Query 45

query = """
/* *** SQL-QUERY-CH03-45 *** */
SELECT FirstName, LastName,
SUM(HoursWorked) AS TotalHoursWorked
FROM EMPLOYEE AS E, ASSIGNMENT AS A
WHERE E.EmployeeNumber = A.EmployeeNumber
GROUP BY LastName, FirstName;
"""
SQLite3_Practice(database_file, query)

  FirstName   LastName  TotalHoursWorked
1      Mary  Abernathy              72.5
2       Tom  Caruthers             100.0
3   Rosalie    Jackson              20.0
4       Tom    Jackson             155.0
5      Mary     Jacobs              90.0
6   Heather      Jones              50.0
7       Ken     Numoto             105.0


In [55]:
# Query 46

query = """
/* *** SQL-QUERY-CH03-46 *** */
SELECT FirstName, LastName, HoursWorked
FROM EMPLOYEE AS E, ASSIGNMENT AS A
WHERE E.EmployeeNumber = A.EmployeeNumber
AND HoursWorked > 50;
"""
SQLite3_Practice(database_file, query)

  FirstName LastName  HoursWorked
1       Tom  Jackson           75
2       Ken   Numoto           55
3       Tom  Jackson           80


In [56]:
# Query 47

query = """
/* *** SQL-QUERY-CH03-47 *** */
SELECT ProjectName, FirstName, LastName, HoursWorked
FROM EMPLOYEE AS E, PROJECT AS P, ASSIGNMENT AS A
WHERE E.EmployeeNumber = A.EmployeeNumber
AND P.ProjectID = A.ProjectID
ORDER BY P.ProjectID, A.EmployeeNumber;
"""
SQLite3_Practice(database_file, query)

                   ProjectName FirstName   LastName  HoursWorked
1         2012 Q3 Product Plan      Mary     Jacobs         30.0
2         2012 Q3 Product Plan       Tom    Jackson         75.0
3         2012 Q3 Product Plan       Ken     Numoto         55.0
4   2012 Q3 Portfolio Analysis       Tom  Caruthers         40.0
5   2012 Q3 Portfolio Analysis      Mary  Abernathy         45.0
6      2012 Q3 Tax Preparation      Mary     Jacobs         25.0
7      2012 Q3 Tax Preparation   Rosalie    Jackson         20.0
8      2012 Q3 Tax Preparation       Tom  Caruthers         45.0
9      2012 Q3 Tax Preparation   Heather      Jones         40.0
10        2012 Q4 Product Plan      Mary     Jacobs         35.0
11        2012 Q4 Product Plan       Tom    Jackson         80.0
12        2012 Q4 Product Plan       Ken     Numoto         50.0
13  2012 Q4 Portfolio Analysis       Tom  Caruthers         15.0
14  2012 Q4 Portfolio Analysis   Heather      Jones         10.0
15  2012 Q4 Portfolio Ana

###///***The SQL JOIN . . . ON Syntax***
An alternative join syntax is the JOIN . . . ON syntax. Consider our query example SQLQUERY-CH03-43 as modified with an ORDER BY clause to become SQL-QUERY-CH03-44.

In [57]:
# Query 44-Recall
# This query uses a join in the WHERE clause:

query = """
/* *** SQL-QUERY-CH03-44 *** */
SELECT FirstName, LastName, HoursWorked
FROM EMPLOYEE, ASSIGNMENT
WHERE EMPLOYEE.EmployeeNumber =
ASSIGNMENT.EmployeeNumber
ORDER BY EMPLOYEE.EmployeeNumber, ProjectID;
"""
SQLite3_Practice(database_file, query)

   FirstName   LastName  HoursWorked
1       Mary     Jacobs         30.0
2       Mary     Jacobs         25.0
3       Mary     Jacobs         35.0
4    Rosalie    Jackson         20.0
5        Tom  Caruthers         40.0
6        Tom  Caruthers         45.0
7        Tom  Caruthers         15.0
8    Heather      Jones         40.0
9    Heather      Jones         10.0
10      Mary  Abernathy         45.0
11      Mary  Abernathy         27.5
12       Tom    Jackson         75.0
13       Tom    Jackson         80.0
14       Ken     Numoto         55.0
15       Ken     Numoto         50.0


In [58]:
# Query 48
# Using the JOIN . . . ON syntax, SQL-QUERY-CH03-44 would be modified as follows to
# become SQL-QUERY-CH03-48:

query = """
/* *** SQL-QUERY-CH03-48 *** */
SELECT FirstName, LastName, HoursWorked
FROM EMPLOYEE JOIN ASSIGNMENT
ON EMPLOYEE.EmployeeNumber =
ASSIGNMENT.EmployeeNumber
ORDER BY EMPLOYEE.EmployeeNumber, ProjectID;
"""
SQLite3_Practice(database_file, query)

   FirstName   LastName  HoursWorked
1       Mary     Jacobs         30.0
2       Mary     Jacobs         25.0
3       Mary     Jacobs         35.0
4    Rosalie    Jackson         20.0
5        Tom  Caruthers         40.0
6        Tom  Caruthers         45.0
7        Tom  Caruthers         15.0
8    Heather      Jones         40.0
9    Heather      Jones         10.0
10      Mary  Abernathy         45.0
11      Mary  Abernathy         27.5
12       Tom    Jackson         75.0
13       Tom    Jackson         80.0
14       Ken     Numoto         55.0
15       Ken     Numoto         50.0


In [59]:
# Query 49
# We can also use the JOIN . . . ON syntax for joins of more than two tables. Here is
# the previous query to combine data for EMPLOYEE, PROJECT, and ASSIGNMENT
# rewritten using the JOIN . . . ON style:

query = """
/* *** SQL-QUERY-CH03-49 *** */
SELECT ProjectName, FirstName, LastName, HoursWorked
FROM EMPLOYEE AS E JOIN ASSIGNMENT AS A
ON E.EmployeeNumber = A.EmployeeNumber
JOIN PROJECT AS P
ON A.ProjectID = P.ProjectID
ORDER BY P.ProjectID, A.EmployeeNumber;
"""
SQLite3_Practice(database_file, query)

                   ProjectName FirstName   LastName  HoursWorked
1         2012 Q3 Product Plan      Mary     Jacobs         30.0
2         2012 Q3 Product Plan       Tom    Jackson         75.0
3         2012 Q3 Product Plan       Ken     Numoto         55.0
4   2012 Q3 Portfolio Analysis       Tom  Caruthers         40.0
5   2012 Q3 Portfolio Analysis      Mary  Abernathy         45.0
6      2012 Q3 Tax Preparation      Mary     Jacobs         25.0
7      2012 Q3 Tax Preparation   Rosalie    Jackson         20.0
8      2012 Q3 Tax Preparation       Tom  Caruthers         45.0
9      2012 Q3 Tax Preparation   Heather      Jones         40.0
10        2012 Q4 Product Plan      Mary     Jacobs         35.0
11        2012 Q4 Product Plan       Tom    Jackson         80.0
12        2012 Q4 Product Plan       Ken     Numoto         50.0
13  2012 Q4 Portfolio Analysis       Tom  Caruthers         15.0
14  2012 Q4 Portfolio Analysis   Heather      Jones         10.0
15  2012 Q4 Portfolio Ana

In [60]:
# Query 05 - Recalled/Edited
# Let’s add a new project, the 2012 Q4 Tax Preparation project run by the
# accounting department, to the PROJECT table as follows:


# Establishing a connection to the SQLite database
conn = sqlite3.connect('marketing_db.sqlite')
cursor = conn.cursor()

"""
/* *** SQL-INSERT-CH03-05 *** */
INSERT INTO PROJECT
(ProjectName, Department, MaxHours, StartDate)
VALUES('2012 Q4 Tax Preparation', 'Accounting',
175.00, '10-DEC-12');
"""

##
# Insert data into PROJECT table
project_data = [
  (1500,'2012 Q4 Tax Preparation', 'Accounting', 175.00, '10-DEC-12', None)
]
for proj in project_data:
  cursor.execute('INSERT INTO PROJECT (ProjectID, ProjectName, Department, MaxHours, StartDate, EndDate) VALUES (?, ?, ?, ?, ?, ?)', proj)


##
# Commit changes and close the connection
conn.commit()
conn.close()

In [61]:
# Query 50
# To see the updated PROJECT table, we use the query:

query = """
/* *** SQL-QUERY-CH03-50 *** */
SELECT * FROM PROJECT;
"""
SQLite3_Practice(database_file, query)

   ProjectID                 ProjectName  Department  MaxHours   StartDate  \
1       1000        2012 Q3 Product Plan   Marketing       135  2012-05-10   
2       1100  2012 Q3 Portfolio Analysis     Finance       120  2012-07-05   
3       1200     2012 Q3 Tax Preparation  Accounting       145  2012-08-10   
4       1300        2012 Q4 Product Plan   Marketing       150  2012-08-10   
5       1400  2012 Q4 Portfolio Analysis     Finance       140  2012-10-05   
6       1500     2012 Q4 Tax Preparation  Accounting       175   10-DEC-12   

      EndDate  
1  2012-06-15  
2  2012-07-25  
3  2012-10-25  
4  2012-09-15  
5        None  
6        None  


In [62]:
# Query 51
# Now, with the new project added to PROJECT, we’ll rerun the previous query on
# EMPLOYEE, ASSIGNMENT, and PROJECT:

query = """
/* *** SQL-QUERY-CH03-51 *** */
SELECT ProjectName, FirstName, LastName, HoursWorked
FROM EMPLOYEE AS E JOIN ASSIGNMENT AS A
ON E.EmployeeNumber = A.EmployeeNumber
JOIN PROJECT AS P
ON A.ProjectID = P.ProjectID
ORDER BY P.ProjectID, A.EmployeeNumber;
"""
SQLite3_Practice(database_file, query)

                   ProjectName FirstName   LastName  HoursWorked
1         2012 Q3 Product Plan      Mary     Jacobs         30.0
2         2012 Q3 Product Plan       Tom    Jackson         75.0
3         2012 Q3 Product Plan       Ken     Numoto         55.0
4   2012 Q3 Portfolio Analysis       Tom  Caruthers         40.0
5   2012 Q3 Portfolio Analysis      Mary  Abernathy         45.0
6      2012 Q3 Tax Preparation      Mary     Jacobs         25.0
7      2012 Q3 Tax Preparation   Rosalie    Jackson         20.0
8      2012 Q3 Tax Preparation       Tom  Caruthers         45.0
9      2012 Q3 Tax Preparation   Heather      Jones         40.0
10        2012 Q4 Product Plan      Mary     Jacobs         35.0
11        2012 Q4 Product Plan       Tom    Jackson         80.0
12        2012 Q4 Product Plan       Ken     Numoto         50.0
13  2012 Q4 Portfolio Analysis       Tom  Caruthers         15.0
14  2012 Q4 Portfolio Analysis   Heather      Jones         10.0
15  2012 Q4 Portfolio Ana

The results shown here are correct, but a surprising result occurs.
- What happened to the new 2012 Q4 Tax Preparation project?
  - The answer is that it does not appear in the join results because its ProjectID value of 1500 had no match in the ASSIGNMENT table.

Nothing is wrong with this result; you just need to be aware that unmatched rows do not appear in the result of a join.

###///***Inner Joins and Outer Joins***
---

The join operation discussed in the previous sections is sometimes referred to as an **equijoin or inner join**.

<br>    

- An **inner join** only displays data from the rows that match based on the join conditions, and as you saw in the last query in the previous section that data can be lost (or at least appear to be lost) when you perform an inner join.
  - In particular, if a row has a value that does not match the WHERE clause condition, that row will not be included in the join result.

    - The 2012 Q4 Tax Preparation project did not appear in the previous join because no row in ASSIGNMENT matched its ProjectID value.
      - **This kind of loss is not always desirable**, so ***`a special type of join, called an outer join, was created to avoid it`***.

<br>    

- **Outer joins** are not part of the SQL-92 specification, but most DBMS products today support them.
  - The specific syntax for the outer join, however, varies by DBMS product.

In [63]:
# Query 52
# Consider the following example and notice the use of the
# JOIN . . . ON syntax—the LEFT keyword is simply added to the SQL query:

query = """
/* *** SQL-QUERY-CH03-52 *** */
SELECT ProjectName, EmployeeNumber, HoursWorked
FROM PROJECT LEFT JOIN ASSIGNMENT
ON PROJECT.ProjectID = ASSIGNMENT.ProjectID;
"""
SQLite3_Practice(database_file, query)

                   ProjectName  EmployeeNumber  HoursWorked
1         2012 Q3 Product Plan             1.0         30.0
2         2012 Q3 Product Plan             8.0         75.0
3         2012 Q3 Product Plan            10.0         55.0
4   2012 Q3 Portfolio Analysis             4.0         40.0
5   2012 Q3 Portfolio Analysis             6.0         45.0
6      2012 Q3 Tax Preparation             1.0         25.0
7      2012 Q3 Tax Preparation             2.0         20.0
8      2012 Q3 Tax Preparation             4.0         45.0
9      2012 Q3 Tax Preparation             5.0         40.0
10        2012 Q4 Product Plan             1.0         35.0
11        2012 Q4 Product Plan             8.0         80.0
12        2012 Q4 Product Plan            10.0         50.0
13  2012 Q4 Portfolio Analysis             4.0         15.0
14  2012 Q4 Portfolio Analysis             5.0         10.0
15  2012 Q4 Portfolio Analysis             6.0         27.5
16     2012 Q4 Tax Preparation          

Notice that the last row of this table appends a null value to the 2012 Q4 Tax Preparation project.

---

<br>

**Right outer joins** operate similarly, except that the `RIGHT` keyword is used, and rows in the table on the right-hand side of the `FROM` clause are included.
- For example, you could join all three tables together with the following right outer join:

In [64]:
# Query 53
# For example, you could join all three tables together with the following
# right outer join:

query = """
/* *** SQL-QUERY-CH03-53 *** */
SELECT ProjectName, HoursWorked, FirstName, LastName
FROM (PROJECT AS P JOIN ASSIGNMENT AS A
ON P.ProjectID = A.ProjectID)
RIGHT JOIN EMPLOYEE AS E
ON A.EmployeeNumber = E.EmployeeNumber
ORDER BY P.ProjectID, A.EmployeeNumber;
"""
SQLite3_Practice(database_file, query)

                   ProjectName  HoursWorked FirstName   LastName
1                         None          NaN   Richard  Bandalone
2                         None          NaN    George      Smith
3                         None          NaN    George      Jones
4                         None          NaN     James     Nestor
5                         None          NaN      Rick      Brown
6         2012 Q3 Product Plan         30.0      Mary     Jacobs
7         2012 Q3 Product Plan         75.0       Tom    Jackson
8         2012 Q3 Product Plan         55.0       Ken     Numoto
9   2012 Q3 Portfolio Analysis         40.0       Tom  Caruthers
10  2012 Q3 Portfolio Analysis         45.0      Mary  Abernathy
11     2012 Q3 Tax Preparation         25.0      Mary     Jacobs
12     2012 Q3 Tax Preparation         20.0   Rosalie    Jackson
13     2012 Q3 Tax Preparation         45.0       Tom  Caruthers
14     2012 Q3 Tax Preparation         40.0   Heather      Jones
15        2012 Q4 Product

In [65]:
# Query 54
# The current data in the EMPLOYEE table looks like this:

query = """
/* *** SQL-QUERY-CH03-54 *** */
SELECT * FROM EMPLOYEE;
"""
SQLite3_Practice(database_file, query)

    EmployeeNumber FirstName   LastName       Department         Phone  \
1                1      Mary     Jacobs   Administration  360-285-8110   
2                2   Rosalie    Jackson   Administration  360-285-8120   
3                3   Richard  Bandalone            Legal  360-285-8210   
4                4       Tom  Caruthers       Accounting  360-285-8310   
5                5   Heather      Jones       Accounting  360-285-8320   
6                6      Mary  Abernathy          Finance  360-285-8410   
7                7    George      Smith  Human Resources  360-285-8510   
8                8       Tom    Jackson       Production  360-287-8610   
9                9    George      Jones       Production  360-287-8620   
10              10       Ken     Numoto        Marketing  360-287-8710   
11              11     James     Nestor      InfoSystems          None   
12              12      Rick      Brown      InfoSystems  360-287-8820   

                        Email  
1    

# SQL and LLM Integration (APplication of Generative AI in DAtabase Management)

In [84]:
import sqlite3

# Path to your existing database
db_file = '/Users/kenechukwuezekwem/Library/CloudStorage/OneDrive-Personal/01_NDSU/WORK/NYU SPS IMC/INTG1-GC 1025 _ Database Management & Modeling/Session02/Session02_Lab/Lab Notes/marketing_db.sqlite'

# Connect to the existing database
conn = sqlite3.connect(db_file)
cursor = conn.cursor()

# Example query to check data from the DEPARTMENT table
cursor.execute('SELECT * FROM DEPARTMENT LIMIT 10;')
print(cursor.fetchall())

# Example query to check data from the EMPLOYEE table
cursor.execute('SELECT * FROM EMPLOYEE LIMIT 10;')
print(cursor.fetchall())

# Example query to check data from the PROJECT table
cursor.execute('SELECT * FROM PROJECT LIMIT 10;')
print(cursor.fetchall())

# Example query to check data from the ASSIGNMENT table
cursor.execute('SELECT * FROM ASSIGNMENT LIMIT 10;')
print(cursor.fetchall())

# Close the connection
conn.close()


[('Administration', 'BC-100-10', 'BLDG01-300', '360-285-8100'), ('Legal', 'BC-200-10', 'BLDG01-200', '360-285-8200'), ('Accounting', 'BC-300-10', 'BLDG01-100', '360-285-8300'), ('Finance', 'BC-400-10', 'BLDG01-140', '360-285-8400'), ('Human Resources', 'BC-500-10', 'BLDG01-180', '360-285-8500'), ('Production', 'BC-600-10', 'BLDG02-100', '360-287-8600'), ('Marketing', 'BC-700-10', 'BLDG02-200', '360-287-8700'), ('InfoSystems', 'BC-800-10', 'BLDG02-270', '360-287-8800')]
[(1, 'Mary', 'Jacobs', 'Administration', '360-285-8110', 'Mary.Jacobs@wpc.com'), (2, 'Rosalie', 'Jackson', 'Administration', '360-285-8120', 'Rosalie.Jackson@wpc.com'), (3, 'Richard', 'Bandalone', 'Legal', '360-285-8210', 'Richard.Bandalone@wpc.com'), (4, 'Tom', 'Caruthers', 'Accounting', '360-285-8310', 'Tom.Caruthers@wpc.com'), (5, 'Heather', 'Jones', 'Accounting', '360-285-8320', 'Heather.Jones@wpc.com'), (6, 'Mary', 'Abernathy', 'Finance', '360-285-8410', 'Mary.Abernathy@wpc.com'), (7, 'George', 'Smith', 'Human Resou

In [98]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
# Load the environment variables from the .env file and # Access your API keys
from dotenv import load_dotenv; import os; load_dotenv(); openai_api_key = os.getenv('OPENAI_API_KEY')

# Template for generating SQL queries
template = """Based on the table schema below, write a SQL query that would answer the user's question:
{schema}

Question: {question}
SQL Query:"""
prompt = ChatPromptTemplate.from_template(template)

# Create a database engine using SQLAlchemy
database_path = "/Users/kenechukwuezekwem/Library/CloudStorage/OneDrive-Personal/01_NDSU/WORK/NYU SPS IMC/INTG1-GC 1025 _ Database Management & Modeling/Session02/Session02_Lab/Lab Notes/marketing_db.sqlite"
connection_string = f"sqlite:///{database_path}"
engine = create_engine(connection_string)

# Initialize the database connection with the engine
db = SQLDatabase(engine)

def get_schema(_):
    return db.get_table_info()

def run_query(query):
    return db.run(query)

# Initialize the model for generating SQL queries
model = ChatOpenAI()  # Replace 'your_api_key' with your actual OpenAI API key

# Build the pipeline for generating and executing SQL queries
sql_response = (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | model.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)

# Invoke the pipeline with a specific question
sql_response.invoke({"question": "How many employees are there?"})

template = """Based on the table schema below, question, sql query, and sql response, write a natural language response:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}"""
prompt_response = ChatPromptTemplate.from_template(template)

full_chain = (
    RunnablePassthrough.assign(query=sql_response).assign(
        schema=get_schema,
        response=lambda x: db.run(x["query"]),
    )
    | prompt_response
    | model
)


In [99]:
def run_ai_SQL_query(question):
    # Generate the SQL query based on the question
    sql_query = sql_response.invoke({"question": question})

    # Execute the SQL query and get the response
    sql_execution_result = db.run(sql_query)

    # Build the context for the natural language response
    context = {
        "schema": get_schema(None),
        "question": question,
        "query": sql_query,
        "response": sql_execution_result
    }

    # Generate the natural language response based on the context
    natural_language_response = full_chain.invoke(context)

    # Print the SQL query, its execution result, and the natural language response with the desired formatting
    print(f"""
SQL Query: 
{sql_query}
SQL Execution Result: {sql_execution_result}

Natural Language Response: 
{natural_language_response}
""")

# Example usage of the function
run_ai_SQL_query("How many employees have a name that starts with E?")



SQL Query: 
SELECT COUNT(*) 
FROM EMPLOYEE 
WHERE FirstName LIKE 'E%'
SQL Execution Result: [(0,)]

Natural Language Response: 
content='There are zero employees whose first name starts with the letter E.' response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 817, 'total_tokens': 830}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_d9767fc5b9', 'finish_reason': 'stop', 'logprobs': None} id='run-021eba95-eb88-489d-8def-1cd2ff92e272-0'



In [100]:
query = """
SELECT *
FROM EMPLOYEE
WHERE FirstName LIKE 'E%';
"""
SQLite3_Practice(database_file, query)

Empty DataFrame
Columns: [EmployeeNumber, FirstName, LastName, Department, Phone, Email]
Index: []


# Practice

In [101]:
# 2	Guo, Hilda	No	-> Find all employees whose first name starts with 'Ma'.

query = """
SELECT *
FROM EMPLOYEE
WHERE FirstName LIKE 'Ma%';
"""
SQLite3_Practice(database_file, query)

   EmployeeNumber FirstName   LastName      Department         Phone  \
1               1      Mary     Jacobs  Administration  360-285-8110   
2               6      Mary  Abernathy         Finance  360-285-8410   

                    Email  
1     Mary.Jacobs@wpc.com  
2  Mary.Abernathy@wpc.com  


In [102]:
run_ai_SQL_query(" Find all employees whose first name starts with 'Ma' ")


SQL Query: 
SELECT * 
FROM EMPLOYEE 
WHERE FirstName LIKE 'Ma%'
SQL Execution Result: [(1, 'Mary', 'Jacobs', 'Administration', '360-285-8110', 'Mary.Jacobs@wpc.com'), (6, 'Mary', 'Abernathy', 'Finance', '360-285-8410', 'Mary.Abernathy@wpc.com')]

Natural Language Response: 
content="The SQL query requested all employees whose first name starts with 'Ma'. The response returned two employees: Mary Jacobs from the Administration department and Mary Abernathy from the Finance department. Both employees' first names start with 'Ma', so they met the criteria specified in the query." response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 876, 'total_tokens': 932}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_d9767fc5b9', 'finish_reason': 'stop', 'logprobs': None} id='run-6ff0c0ff-ec8b-47fe-9be2-b0441f7d1f3c-0'

